In [19]:
import pandas
import json
import numpy as np

In [20]:

class PM_Data:
    def __init__(self, file):
        self.pm_file = file

    def load(self, pollutant_standard = 'PM25 24-hour 2012' ):
        with open(self.pm_file) as f:
            pm_data =json.load(f)['Data']
            #pd = pandas.DataFrame(pm_data)
            #  take only 1 measurement - daily. according to a specific standard that all station has
            return pm_data


In [21]:
def create_daily_avarage_pm_data(pm_data):
    pm_per_date = {}
    for pm_meas in pm_data:
        date = pm_meas['date_gmt']
        station_num = pm_meas['site_number'] # or by site number!
        if date not in pm_per_date.keys():
            pm_per_date[date] = {}
        if station_num not in pm_per_date[date].keys():
            pm_per_date[date][station_num] = []
        pm_per_date[date][station_num].append(pm_meas)
    
    for date in list(pm_per_date):
        for station_num in list(pm_per_date[date]):
            pm_meas = pm_per_date[date][station_num][0]
            samples = [i['sample_measurement'] for i in pm_per_date[date][station_num] if i['sample_measurement'] is not None ]
            if len(samples)>0:
                ave = np.mean(samples)
                pm_per_date[date][station_num]= pm_meas
                pm_per_date[date][station_num]['sample_measurement'] = ave
                pm_per_date[date][station_num]['std'] = np.std(samples)
            else:
                del pm_per_date[date][station_num]
    
    return pm_per_date


In [22]:
pm_data_file = '/home/adiraz/adiraz/misr_data/la_pm_2021_all.json'
pd = PM_Data(pm_data_file)
pm_data = pd.load()

out = create_daily_avarage_pm_data(pm_data)
import copy
out_only_meas = copy.deepcopy(out)
for date in out_only_meas.keys():
    for station_num in out_only_meas[date].keys():
        out_only_meas[date][station_num] = (out_only_meas[date][station_num]['sample_measurement'],out_only_meas[date][station_num]['std'])


In [8]:
pm_data[0]

KeyError: 0

# View Raw PM DATA

In [23]:
import pandas
pandas.set_option('display.max_columns', None)
#pandas.set_option("max_rows", None)
p = pandas.DataFrame(pm_data)
p.head(100)

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,date_local,time_local,date_gmt,time_gmt,sample_measurement,units_of_measure,units_of_measure_code,sample_duration,sample_duration_code,sample_frequency,detection_limit,uncertainty,qualifier,method_type,method,method_code,state,county,date_of_last_change,cbsa_code,std
0,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-01,00:00,2021-01-01,08:00,64.570588,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,IH - Fireworks.,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,50.429319
1,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-01,01:00,2021-01-01,09:00,126.500000,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,IH - Fireworks.,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,NaN
2,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-01,02:00,2021-01-01,10:00,109.000000,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,IH - Fireworks.,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,NaN
3,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-01,03:00,2021-01-01,11:00,115.500000,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,IH - Fireworks.,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,NaN
4,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-01,04:00,2021-01-01,12:00,119.800000,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,IH - Fireworks.,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-04,23:00,2021-01-05,07:00,10.000000,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,None,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,NaN
96,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-05,00:00,2021-01-05,08:00,13.200000,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,None,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,NaN
97,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-05,01:00,2021-01-05,09:00,13.700000,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,None,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,NaN
98,06,037,4004,88101,3,33.79236,-118.17533,WGS84,PM2.5 - Local Conditions,2021-01-05,02:00,2021-01-05,10:00,14.200000,Micrograms/cubic meter (LC),105,1 HOUR,1,HOURLY,5.0,None,None,FEM,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,170,California,Los Angeles,2021-06-22,31080,NaN


# My Daily Avarge based on the hourly measurments

In [23]:
pandas.DataFrame(out_only_meas)


,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2021-01-19,2021-01-20,2021-01-21,2021-01-22,2021-01-23,2021-01-24,2021-01-25,2021-01-26,2021-01-27,2021-01-28,2021-01-29,2021-01-30,2021-01-31,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-12,2021-02-13,2021-02-14,2021-02-15,2021-02-16,2021-02-17,2021-02-18,2021-02-19,2021-02-20,2021-02-21,2021-02-22,2021-02-23,2021-02-24,2021-02-25,2021-02-26,2021-02-27,2021-02-28,2021-03-01,2021-03-02,2021-03-03,2021-03-04,2021-03-05,2021-03-06,2021-03-07,2021-03-08,2021-03-09,2021-03-10,2021-03-11,2021-03-12,2021-03-13,2021-03-14,2021-03-15,2021-03-16,2021-03-17,2021-03-18,2021-03-19,2021-03-20,2021-03-21,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29,2021-03-30,2021-03-31,2021-04-01,2021-04-02,2021-04-03,2021-04-04,2021-04-05,2021-04-06,2021-04-07,2021-04-08,2021-04-09,2021-04-10,2021-04-11,2021-04-12,2021-04-13,2021-04-14,2021-04-15,2021-04-16,2021-04-17,2021-04-18,2021-04-19,2021-04-20,2021-04-21,2021-04-22,2021-04-23,2021-04-24,2021-04-25,2021-04-26,2021-04-27,2021-04-28,2021-04-29,2021-04-30,2021-05-01,2021-05-02,2021-05-03,2021-05-04,2021-05-05,2021-05-06,2021-05-07,2021-05-08,2021-05-09,2021-05-10,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-15,2021-05-16,2021-05-17,2021-05-18,2021-05-19,2021-05-20,2021-05-21,2021-05-22,2021-05-23,2021-05-24,2021-05-25,2021-05-26,2021-05-27,2021-05-28,2021-05-29,2021-05-30,2021-05-31,2021-06-01,2021-06-02,2021-06-03,2021-06-04,2021-06-05,2021-06-06,2021-06-07,2021-06-08,2021-06-09,2021-06-10,2021-06-11,2021-06-12,2021-06-13,2021-06-14,2021-06-15,2021-06-16,2021-06-17,2021-06-18,2021-06-19,2021-06-20,2021-06-21,2021-06-22,2021-06-23,2021-06-24,2021-06-25,2021-06-26,2021-06-27,2021-06-28,2021-06-29,2021-06-30,2021-07-01,2021-07-02,2021-07-03,2021-07-04,2021-07-05,2021-07-06,2021-07-07,2021-07-08,2021-07-09,2021-07-10,2021-07-11,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-17,2021-07-18,2021-07-19,2021-07-20,2021-07-21,2021-07-22,2021-07-23,2021-07-24,2021-07-25,2021-07-26,2021-07-27,2021-07-28,2021-07-29,2021-07-30,2021-07-31,2021-08-01,2021-08-02,2021-08-03,2021-08-04,2021-08-05,2021-08-06,2021-08-07,2021-08-08,2021-08-09,2021-08-10,2021-08-11,2021-08-12,2021-08-13,2021-08-14,2021-08-15,2021-08-16,2021-08-17,2021-08-18,2021-08-19,2021-08-20,2021-08-21,2021-08-22,2021-08-23,2021-08-24,2021-08-25,2021-08-26,2021-08-27,2021-08-28,2021-08-29,2021-08-30,2021-08-31,2021-09-01,2021-09-02,2021-09-03,2021-09-04,2021-09-05,2021-09-06,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,2021-09-14,2021-09-15,2021-09-16,2021-09-17,2021-09-18,2021-09-19,2021-09-20,2021-09-21,2021-09-22,2021-09-23,2021-09-24,2021-09-25,2021-09-26,2021-09-27,2021-09-28,2021-09-29,2021-09-30,2021-10-01,2021-10-02,2021-10-03,2021-10-04,2021-10-05,2021-10-06,2021-10-07,2021-10-08,2021-10-09,2021-10-10,2021-10-11,2021-10-12,2021-10-13,2021-10-14,2021-10-15,2021-10-16,2021-10-17,2021-10-18,2021-10-19,2021-10-20,2021-10-21,2021-10-22,2021-10-23,2021-10-24,2021-10-25,2021-10-26,2021-10-27,2021-10-28,2021-10-29,2021-10-30,2021-10-31,2021-11-01,2021-11-02,2021-11-03,2021-11-04,2021-11-05,2021-11-06,2021-11-07,2021-11-08,2021-11-09,2021-11-10,2021-11-11,2021-11-12,2021-11-13,2021-11-14,2021-11-15,2021-11-16,2021-11-17,2021-11-18,2021-11-19,2021-11-20,2021-11-21,2021-11-22,2021-11-23,2021-11-24,2021-11-25,2021-11-26,2021-11-27,2021-11-28,2021-11-29,2021-11-30,2021-12-01,2021-12-02,2021-12-03,2021-12-04,2021-12-05,2021-12-06,2021-12-07,2021-12-08,2021-12-09,2021-12-10,2021-12-11,2021-12-12,2021-12-13,2021-12-14,2021-12-15,2021-12-16,2021-12-17,2021-12-18,2021-12-19,2021-12-20,2021-12-21,2021-12-22,2021-12-23,2021-12-24,2021-12-25,2021-12-26,2021-12-27,2021-12-28,2021-12-29,2021-1

# Sample frequency for each station in the data 

## show sample freq for each meas

In [24]:

pandas.DataFrame([i['sample_frequency'] for i in pm_data if i['sample_duration'] == "24 HOUR"])

,0
0,EVERY 6TH DAY
1,EVERY 6TH DAY
2,EVERY 6TH DAY
3,EVERY 6TH DAY
4,EVERY 6TH DAY
...,...
2188,EVERY 3RD DAY
2189,EVERY 3RD DAY
2190,EVERY 3RD DAY
2191,EVERY 3RD DAY


## count sample frequencies

In [29]:
p['sample_frequency'].value_counts()

HOURLY           26280
EVERY DAY         1460
EVERY 6TH DAY      367
EVERY 3RD DAY      366
Name: sample_frequency, dtype: int64

## Check how many stations have hourly meas??

In [30]:
p[p['sample_frequency'] == 'HOURLY']['site_number'].unique()

array(['4004', '9033', '4008'], dtype=object)

In [12]:
p[p['sample_frequency'] == 'EVERY DAY']['site_number'].unique()

array(['1302', '4008', '4004', '1103'], dtype=object)

In [13]:
p[p['sample_frequency'] == 'EVERY 6TH DAY']['site_number'].unique()

array(['0002', '1602', '1103'], dtype=object)

In [14]:
p[p['sample_frequency'] == 'EVERY 3RD DAY']['site_number'].unique()

array(['4002', '1201', '2005'], dtype=object)

In [ ]:
## Show freq for each station

In [15]:
#p.groupby(['site_number',"sample_frequency"]).size()
p.groupby(['site_number',"sample_duration"]).size()

site_number  sample_duration
0002         24 HOUR             122
1103         24 HOUR             427
1201         24 HOUR             122
1302         24 HOUR             365
1602         24 HOUR             183
2005         24 HOUR             122
4002         24 HOUR             122
4004         1 HOUR             8760
             24 HOUR             365
4008         1 HOUR             8760
             24 HOUR             365
9033         1 HOUR             8760
dtype: int64

In [16]:
#p[p['site_number']=='0002']

# Check Daily Avarages data

In [24]:
pm_daily_data_file = '/home/adiraz/adiraz/misr_data/la_pm_2021_daily.json'
pdaily = PM_Data(pm_daily_data_file)
pm_data_d = pdaily.load()

pm_data_daily = pandas.DataFrame(pm_data_d)
pm_data_daily.head(10)

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,sample_duration,pollutant_standard,date_local,units_of_measure,event_type,observation_count,observation_percent,validity_indicator,arithmetic_mean,first_max_value,first_max_hour,aqi,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 Annual 1997,2021-12-31,Micrograms/cubic meter (LC),No Events,1,100.0,Y,4.200000,4.2,0,18.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
1,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 1997,2021-12-31,Micrograms/cubic meter (LC),No Events,1,100.0,Y,4.200000,4.2,0,18.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
2,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 Annual 2012,2021-12-31,Micrograms/cubic meter (LC),No Events,1,100.0,Y,4.200000,4.2,0,18.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
3,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 2012,2021-12-31,Micrograms/cubic meter (LC),No Events,1,100.0,Y,4.200000,4.2,0,18.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
4,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 Annual 2006,2021-12-31,Micrograms/cubic meter (LC),No Events,1,100.0,Y,4.200000,4.2,0,18.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
5,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 2006,2021-12-31,Micrograms/cubic meter (LC),No Events,1,100.0,Y,4.200000,4.2,0,18.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
6,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,1,1 HOUR,None,2021-12-31,Micrograms/cubic meter (LC),No Events,24,100.0,Y,4.291667,14.0,2,NaN,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
7,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 Annual 1997,2021-12-30,Micrograms/cubic meter (LC),No Events,1,100.0,Y,3.000000,3.0,0,13.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
8,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 1997,2021-12-30,Micrograms/cubic meter (LC),No Events,1,100.0,Y,3.000000,3.0,0,13.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
9,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 Annual 2012,20

In [133]:
pm_data_daily.groupby(['site_number',"pollutant_standard"]).size()

site_number  pollutant_standard
0002         PM25 24-hour 1997     120
             PM25 24-hour 2006     120
             PM25 24-hour 2012     120
             PM25 Annual 1997      120
             PM25 Annual 2006      120
             PM25 Annual 2012      120
1103         PM25 24-hour 1997     422
             PM25 24-hour 2006     422
             PM25 24-hour 2012     422
             PM25 Annual 1997      422
             PM25 Annual 2006      422
             PM25 Annual 2012      422
1201         PM25 24-hour 1997     120
             PM25 24-hour 2006     120
             PM25 24-hour 2012     120
             PM25 Annual 1997      120
             PM25 Annual 2006      120
             PM25 Annual 2012      120
1302         PM25 24-hour 1997     349
             PM25 24-hour 2006     349
             PM25 24-hour 2012     349
             PM25 Annual 1997      349
             PM25 Annual 2006      349
             PM25 Annual 2012      349
1602         PM25 24-hour 1997  

In [145]:
#pm_data_daily[pm_data_daily['site_number'] == '4004' and pm_data_daily['sample_duration'] == '1 HOUR']

#pm_data_daily.loc[(pm_data_daily['site_number']== '4004') & (pm_data_daily['sample_duration'] == '24 HOUR')]

pm_data_daily.loc[(pm_data_daily['pollutant_standard']== 'PM25 24-hour 2012')]# & (pm_data_daily['date_local'] =='2021-01-21')][['site_number','date_local']]

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,sample_duration,pollutant_standard,date_local,units_of_measure,event_type,observation_count,observation_percent,validity_indicator,arithmetic_mean,first_max_value,first_max_hour,aqi,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
3,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 2012,2021-12-31,Micrograms/cubic meter (LC),No Events,1,100.0,Y,4.2,4.2,0,18.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
10,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 2012,2021-12-30,Micrograms/cubic meter (LC),No Events,1,100.0,Y,3.0,3.0,0,13.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
17,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 2012,2021-12-29,Micrograms/cubic meter (LC),No Events,1,100.0,Y,2.0,2.0,0,8.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
24,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 2012,2021-12-28,Micrograms/cubic meter (LC),No Events,1,100.0,Y,2.2,2.2,0,9.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
31,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 2012,2021-12-27,Micrograms/cubic meter (LC),No Events,1,100.0,Y,2.1,2.1,0,9.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230,06,037,0002,88101,21,34.136500,-117.923910,WGS84,PM2.5 - Local Conditions,7,24 HOUR,PM25 24-hour 2012,2021-01-25,Micrograms/cubic meter (LC),No Events,1,100.0,Y,2.6,2.6,0,11.0,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,Azusa,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-07-01
20236,06,037,0002,88101,21,34.136500,-117.923910,WGS84,PM2.5 - Local Conditions,7,24 HOUR,PM25 24-hour 2012,2021-01-19,Micrograms/cubic meter (LC),No Events,1,100.0,Y,13.0,13.0,0,53.0,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,Azusa,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-07-01
20242,06,037,0002,88101,21,34.136500,-117.923910,WGS84,PM2.5 - Local Conditions,7,24 HOUR,PM25 24-hour 2012,2021-01-13,Micrograms/cubic meter (LC),No Events,1,100.0,Y,4.8,4.8,0,20.0,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,Azusa,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-07-01
20248,06,037,0002,88101,21,34.136500,-117.923910,WGS84,PM2.5 - Local Conditions,7,24 HOUR,PM25 24-hour 2012,2021-01-07,Micrograms/cubic meter (LC),No Events,1,100.0,Y,11.0,11.0,0,46.0,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,Azusa,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-07-01


In [ ]:
def create_daily_avarage_pm_data(pm_data):
    pm_per_date = {}
    for pm_meas in pm_data:
        date = pm_meas['date_local']
        station_num = pm_meas['site_number'] # or by site number!
        if date not in pm_per_date.keys():
            pm_per_date[date] = {}
        if station_num not in pm_per_date[date].keys():
            pm_per_date[date][station_num] = []
        pm_per_date[date][station_num].append(pm_meas)
    
    for date in list(pm_per_date):
        for station_num in list(pm_per_date[date]):
            pm_meas = pm_per_date[date][station_num][0]
            samples = [i['sample_measurement'] for i in pm_per_date[date][station_num] if i['sample_measurement'] is not None ]
            if len(samples)>0:
                ave = np.mean(samples)
                pm_per_date[date][station_num]= pm_meas
                pm_per_date[date][station_num]['sample_measurement'] = ave
                pm_per_date[date][station_num]['std'] = np.std(samples)
            else:
                del pm_per_date[date][station_num]
    
    return pm_per_date


In [109]:
pm_data_daily.groupby(['site_number',"date_local"]).size()


site_number  sample_duration
0002         24 HOUR             720
1103         24 HOUR            2532
1201         24 HOUR             720
1302         24 HOUR            2094
1602         24 HOUR            1098
2005         24 HOUR             714
4002         24 HOUR             714
4004         1 HOUR              360
             24 HOUR            2136
             24-HR BLK AVG      2136
4008         1 HOUR              340
             24 HOUR            2160
             24-HR BLK AVG      2016
9033         1 HOUR              363
             24-HR BLK AVG      2154
dtype: int64

# Compare hourly and daily results:

## Daily Average ( 2020-01-04 station 9033)

In [107]:
p_date = pm_data_daily[pm_data_daily["date_local"] == '2021-01-04'] 
p_date_24 = p_date[p_date['pollutant_standard'] == 'PM25 24-hour 2012']
p_date_site = p_date_24[p_date_24['site_number'] == '9033']
p_date_site
# p_date_site['sample_measurement'], p_date_site['time_gmt']

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,sample_duration,pollutant_standard,date_local,units_of_measure,event_type,observation_count,observation_percent,validity_indicator,arithmetic_mean,first_max_value,first_max_hour,aqi,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
2275,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,24-HR BLK AVG,PM25 24-hour 2012,2021-02-04,Micrograms/cubic meter (LC),No Events,1,100.0,Y,6.2,6.2,0,26.0,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02


## Same day hourly samples:

In [102]:
p_date = p[p["date_gmt"] == '2021-01-04']
p_date_site = p_date[p_date['site_number'] == '9033']
p_date_site[['time_gmt','sample_measurement']]

,time_gmt,sample_measurement
8824,00:00,8.416667
8825,01:00,4.000000
8826,02:00,6.000000
8827,03:00,6.000000
8828,04:00,9.000000
8829,05:00,12.000000
8830,06:00,20.000000
8831,07:00,16.000000
8832,08:00,5.000000
8833,09:00,5.000000


In [104]:
p['sample_measurement'].max()

292.7

## check dates in daily avarages

In [73]:
dates = pm_data_daily[pm_data_daily['site_number'] == '9033']['date_local']
to_dates =pandas.to_datetime(dates)
to_dates.max()

Timestamp('2021-12-31 00:00:00')

# Perpare data


I decided to go with daily data since pm is usually measured daily, sample duration is usually daily according to this explanation: https://aqs.epa.gov/aqsweb/airdata/FileFormats.html ("Daily Summary Files")


In [11]:

pm_daily_data_file = '/home/adiraz/adiraz/misr_data/la_pm_2021.json'
pdaily = PM_Data(pm_daily_data_file)
pm_data_d = pdaily.load()

pm_data_daily = pandas.DataFrame(pm_data_d)
pm_data_daily.head(10)

#  take only 1 measurement - daily. according to a specific standard that all station has
pm_daily_clean = pm_data_daily.loc[(pm_data_daily['pollutant_standard']== 'PM25 24-hour 2012')]# & (pm_data_daily['date_local'] =='2021-01-21')][['site_number','date_local']]

KeyError: 'pollutant_standard'

In [12]:
pm_daily_data_file = '/home/adiraz/adiraz/misr_data/la_pm_2021_daily.json'

with open(pm_daily_data_file) as f:
    pm_data =json.load(f)['Data']
    pm_data_daily = pandas.DataFrame(pm_data)

In [13]:
pm_data_daily


,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,...,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
1,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,...,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
2,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,...,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
3,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,...,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
4,06,037,9033,88101,1,34.669739,-118.130511,NAD83,PM2.5 - Local Conditions,X,...,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,Lancaster-Division Street,"43301 DIVISION ST., LANCASTER, CA",California,Los Angeles,Lancaster,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-05-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20252,06,037,0002,88101,21,34.136500,-117.923910,WGS84,PM2.5 - Local Conditions,7,...,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,Azusa,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-07-01
20253,06,037,0002,88101,21,34.136500,-117.923910,WGS84,PM2.5 - Local Conditions,7,...,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,Azusa,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-07-01
20254,06,037,0002,88101,21,34.136500,-117.923910,WGS84,PM2.5 - Local Conditions,7,...,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,Azusa,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-07-01
20255,06,037,0002,88101,21,34.136500,-117.923910,WGS84,PM2.5 - Local Conditions,7,...,143,R & P Model 2000 PM-2.5 Air Sampler w/VSCC - G...,Azusa,"803 N. LOREN AVE., AZUSA",California,Los Angeles,Azusa,31080,"Los Angeles-Long Beach-Anaheim, CA",2022-07-01
